In [1]:
import pandas as pd
import os

In [37]:
path = "/Users/cmblir/Python/Lunch-Chat-Bot/kakao-map-collection/collect data/transfrom data/"
filepath = os.listdir(path)

In [47]:
df = pd.read_csv(path + "강남구.csv", sep="|")

In [40]:
default_df = pd.DataFrame(columns = list(df.columns))

In [41]:
for file in filepath:
    try:
        df = pd.read_csv(path + file, sep = "|")
    except UnicodeDecodeError:
        print(file)
        df = pd.read_csv(path + file, sep = "|", encoding="cp949")
    df = df.drop(["Unnamed: 7"], axis="columns")
    default_df = default_df.append(df)

/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_11839/3693255687.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  default_df = default_df.append(df)
/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_11839/3693255687.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  default_df = default_df.append(df)
/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_11839/3693255687.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  default_df = default_df.append(df)
/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_11839/3693255687.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  default_df = default_df.app

In [46]:
default_df["맛집명"].duplicated().value_counts()

False    11712
True       297
Name: 맛집명, dtype: int64